## Proyecto Capstone - La batalla de los Vecindarios (Semana 2)
Proyecto Final de Certificacion en Ciencia de Datos por IBM/Coursera con el apoyo de TalentoTech El Salvador

Presentado por: Cristian Parada

## Tabla de contenidos.
* [Introducción: Problema de negocio.](#introduccion)
* [Datos.](#datos)
* [Metodologia](#metodologia)
* [Analisis](#analisis)
* [Resultados y Discusion](#resultados)
* [Conclusion](#conclusion)

<h2>Introducción: Problema de negocio.</h2> <a name="introduccion"></a>

<p>En este proyecto vamos a tratar de encontrar el mejor apartamento en alquiler debido a una oferta de trabajo. Este proyecto está dirigido a todos los que puedan estar interesados en saber más acerca de los precios de alquiler de casas y  apartamentos, las estadísticas de las ciudades o simplemente quieren mudarse a San Salvador, El Salvador, en America Central.</p>

<p>Como nuestra oferta de trabajo es para una dirección específica, hemos seleccionado algunas ciudades cercanas para vivir. A pesar de que hemos seleccionado estas ciudades, todavía hay una gran cantidad de apartamentos y casas en alquiler. Tenemos algunas exigencias con respecto a los lugares cercanos, y que se tendrán en cuenta para elegir nuestra casa o apartamento.</p>

<p>Utilizaremos nuestras técnicas en ciencia de datos para generar las casas o apartamentos más prometedores para vivir. En el proceso se tendrán en cuenta datos económicos y características sociales de las ciudades que conforman el Gran San Salvador.</p>


<h2>Datos.</h2> <a name="datos"></a>

A partir de la definición de nuestro problema, los factores que influirán en nuestra decisión son:
<p>-El apartamento debe tener al menos 2 dormitorios.</p>
<p>-El valor mensual del alquiler no puede superar USD 500.00. </p>
<p>-Tiene que estar al menos a 15 km del lugar de trabajo.</p>
<p>-Tiene que estar cerca de al menos 5 km de un gimnasio y un mercado o supermercado.</p>
<p>-Características sociales y económicas del lugar según los datos disponibles.<p>
<p>-Decidimos seleccionar algunos barrios de los alrededores de nuestro lugar de trabajo para extraer nuestros datos.</p>

Se necesitarán las siguientes fuentes de datos para extraer/generar la información necesaria:
<p>-Las coordenadas del lugar de trabajo se obtendrán mediante geocodificador.</p>
<p>-Los datos económicos y de ubicación de las viviendas de alquiler alrededor del lugar de trabajo se obtendrán mediante webscraping de varios portales en línea.</p>
<p>-Los locales cercanos alrededor de cada barrio se obtendrán mediante la API de Foursquare.</p>

<h3><p>Cargando nuestras bibliotecas y los datos de las casas y apartamentos</p></h3>

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import geocoder
import random
import folium
import unicodedata2
from geopy.geocoders import Nominatim
from haversine import haversine
import requests
import folium
from sklearn.cluster import KMeans
print('Librerias importadas exitosamente!')

Librerias importadas exitosamente!


In [7]:
# Antes de empezar, definamos nuestros factores básicos.
starting_point = 'Edificio Insigne, Av. Las Magnolias San Salvador, El Salvador'
max_rent_value = 500
min_bedrooms = 2
min_dist_workplace = 15
min_dist_gym = 8
min_dist_mkt = 5
print('Factores basicos, grabados con exito!')

Factores basicos, grabados con exito!


In [8]:
dataset = pd.read_csv('alquileres.csv')
print('Set de datos cargado con exito!')

Set de datos cargado con exito!


In [9]:
dataset.head()

,codigo,tipo_alquiler,habitaciones,precio_alquiler,Neighborhood,Municipio,Departamento,Latitude,Longitude
0,1,Apartamento,1,450,Col. Libertad,San Salvador,San Salvador,13.7225,-89.2097
1,2,Habitacion,1,90,Calle San Ernesto por Corvisa,San Salvador,San Salvador,13.6814,-89.2011
2,3,Habitacion,1,190,Col. Miramonte,San Salvador,San Salvador,13.7079,-89.2142
3,4,Habitacion,1,200,Res. Buena vista,Santa Tecla,La Libertad,13.6896,-89.2854
4,5,Apartamento,1,400,Res. Buena vista,Santa Tecla,La Libertad,13.6896,-89.2854


In [10]:
dataset.shape

(102, 9)

Validando de que todas las propiedades son casas y apartamentos.

In [11]:
dataset['tipo_alquiler'].value_counts()

Casa           80
Habitacion     12
Apartamento    10
Name: tipo_alquiler, dtype: int64

Ahora, vamos a obtener las coordenadas de nuestro lugar de trabajo para obtener los sitios de vivienda.

In [12]:
work_location = geocoder.arcgis(starting_point).latlng

In [13]:
work_location

[13.714951333129749, -89.21544013829029]

Obteniendo las coordenadas graficas de San Salvador.

In [14]:
address = 'San Salvador, SV'
geolocator = Nominatim(user_agent="sv_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Las coordenadas geograficas de San Salvador son {}, {}.'.format(latitude, longitude))

Las coordenadas geograficas de San Salvador son 13.6989939, -89.1914249.


Asegurémonos de que se rellenan todos los valores de latitud y longitud.

In [15]:
dataset = dataset.loc[(dataset['Latitude'] != '-') & (dataset['Longitude'] != '-')]
dataset.shape

(102, 9)

Ahora tenemos que medir la distancia entre cada casa, habitacion y apartamento y nuestro lugar de trabajo. En primer lugar, vamos a crear una función para combinar la latitud y la longitud en una sola fila.

In [16]:
def get_location(dataframe, name_latitude, name_longitude):
    df = dataframe.copy()
    df['location'] = df[name_latitude].astype(str) + ',' + df[name_longitude].astype(str)
    df['location'] = df['location'].apply(lambda x: x.split(','))
    df['location'] = df['location'].apply(lambda x: [float(i) for i in x])
    return df

In [17]:
dataset = get_location(dataset, name_latitude='Latitude', name_longitude='Longitude')

In [18]:
dataset.head()

,codigo,tipo_alquiler,habitaciones,precio_alquiler,Neighborhood,Municipio,Departamento,Latitude,Longitude,location
0,1,Apartamento,1,450,Col. Libertad,San Salvador,San Salvador,13.7225,-89.2097,"[13.7225, -89.2097]"
1,2,Habitacion,1,90,Calle San Ernesto por Corvisa,San Salvador,San Salvador,13.6814,-89.2011,"[13.6814, -89.2011]"
2,3,Habitacion,1,190,Col. Miramonte,San Salvador,San Salvador,13.7079,-89.2142,"[13.7079, -89.2142]"
3,4,Habitacion,1,200,Res. Buena vista,Santa Tecla,La Libertad,13.6896,-89.2854,"[13.6896, -89.2854]"
4,5,Apartamento,1,400,Res. Buena vista,Santa Tecla,La Libertad,13.6896,-89.2854,"[13.6896, -89.2854]"


Vamos a utilizar la librería Haversine para medir la distancia entre puntos utilizando la latitud y longitud de los mismos. La distancia viene dada en km.

In [19]:
dataset['distance_from_workplace'] = dataset['location'].apply(lambda x: haversine(work_location, x))

In [20]:
dataset.head()

,codigo,tipo_alquiler,habitaciones,precio_alquiler,Neighborhood,Municipio,Departamento,Latitude,Longitude,location,distance_from_workplace
0,1,Apartamento,1,450,Col. Libertad,San Salvador,San Salvador,13.7225,-89.2097,"[13.7225, -89.2097]",1.043567
1,2,Habitacion,1,90,Calle San Ernesto por Corvisa,San Salvador,San Salvador,13.6814,-89.2011,"[13.6814, -89.2011]",4.039611
2,3,Habitacion,1,190,Col. Miramonte,San Salvador,San Salvador,13.7079,-89.2142,"[13.7079, -89.2142]",0.795436
3,4,Habitacion,1,200,Res. Buena vista,Santa Tecla,La Libertad,13.6896,-89.2854,"[13.6896, -89.2854]",8.066393
4,5,Apartamento,1,400,Res. Buena vista,Santa Tecla,La Libertad,13.6896,-89.2854,"[13.6896, -89.2854]",8.066393


Ahora, establezcamos un radio de 15 km alrededor de nuestro lugar de trabajo y filtremos todos los lugares de alquiler que se encuentren dentro de él.

In [21]:
dataset = dataset.loc[dataset['distance_from_workplace'] <= min_dist_workplace]

In [22]:
dataset.shape

(56, 11)

In [23]:
dataset.head()

,codigo,tipo_alquiler,habitaciones,precio_alquiler,Neighborhood,Municipio,Departamento,Latitude,Longitude,location,distance_from_workplace
0,1,Apartamento,1,450,Col. Libertad,San Salvador,San Salvador,13.7225,-89.2097,"[13.7225, -89.2097]",1.043567
1,2,Habitacion,1,90,Calle San Ernesto por Corvisa,San Salvador,San Salvador,13.6814,-89.2011,"[13.6814, -89.2011]",4.039611
2,3,Habitacion,1,190,Col. Miramonte,San Salvador,San Salvador,13.7079,-89.2142,"[13.7079, -89.2142]",0.795436
3,4,Habitacion,1,200,Res. Buena vista,Santa Tecla,La Libertad,13.6896,-89.2854,"[13.6896, -89.2854]",8.066393
4,5,Apartamento,1,400,Res. Buena vista,Santa Tecla,La Libertad,13.6896,-89.2854,"[13.6896, -89.2854]",8.066393


Ahora que tenemos nuestro conjunto de datos limpio, vamos a crear un mapa para visualizar la ubicación de cada apartamento en un radio de 15 km de nuestro lugar de trabajo.

In [70]:
# Instanciar el mapa
map_city = folium.Map(location=[work_location[0], work_location[1]], zoom_start=14)

# añadir los marcadores de cada casa, habitacion y apartamento
for lat, lng, label, distance in zip(dataset['Latitude'], dataset['Longitude'], dataset['codigo'], dataset['distance_from_workplace']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=f'code={label}\ndistance={round(distance, 15)}km',
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_city)

# añadir el marcador de lugar de trabajo
folium.Marker(
    [work_location[0], work_location[1]],
    radius=5,
    tooltip='Lugar de Trabajo(Edificio Insigne, Av. Las Magnolias San Salvador)',
    icon=folium.Icon(icon='briefcase', color="green")
).add_to(map_city)

# Agregando los 15 kms de radio.
folium.Circle([work_location[0], work_location[1]],
                    radius=5000,
                    color='red'
                   ).add_to(map_city)

map_city

Ahora, para complementar nuestra visualización, vamos a crear un mapa de calor para ver cómo se distribuyen nuestros tipos de alquiler.

In [71]:
#Importando la libreria
from folium.plugins import HeatMap

#Trayendo la lista de Lat y Lng
lat = dataset['Latitude'] 
lng = dataset['Longitude']

#Creando el mapa de calor
map = folium.Map(
    location=[13.6989939, -89.1914249],
    tiles='cartodbpositron',  #cartodbdark_matter
    zoom_start=11
)
HeatMap(list(zip(lat, lng))).add_to(map)

# Añadir el marcador de lugar de trabajo
folium.Marker(
    [work_location[0], work_location[1]],
    radius=5,
    tooltip='Lugar de Trabajo(Edificio Insigne, Av. Las Magnolias San Salvador)',
    icon=folium.Icon(icon='briefcase', color="green")
).add_to(map)

# Agregando los 15 kms de radio.
folium.Circle([work_location[0], work_location[1]],
                    radius=5000,
                    color='red'
                   ).add_to(map)
map

Ahora que tenemos las propiedades mapeadas, vamos a extraer los locales cercanos a través de cada vecindario.

## Foursquare

Vamos a rellenar nuestras credenciales para utilizar la API de foursquare.

In [26]:
# Utilizar la API de Foursquare para explorar los barrios y segmentarlos.
# Definir credenciales de Foursquare y versión
CLIENT_ID = 'JH4PNBUYQL4VQOVJNTCSWLFDQBPNVZTSWKC5UTWXNKNJMH20' # your Foursquare ID
CLIENT_SECRET = 'KIP3WV5SAR3G2XNAKEVKKAXFRK2KRWYM1XEF4CPGCJYWLMA5' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

print('Mis Credenciales:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Mis Credenciales:
CLIENT_ID: JH4PNBUYQL4VQOVJNTCSWLFDQBPNVZTSWKC5UTWXNKNJMH20
CLIENT_SECRET:KIP3WV5SAR3G2XNAKEVKKAXFRK2KRWYM1XEF4CPGCJYWLMA5


Ahora vamos a ver cuáles son los vecindarios para extraer los locales cercanos.

In [27]:
dataset['Neighborhood'].value_counts()

Col. Miralvalle                                          3
Res. Villa Constitucion                                  2
Col. Miramonte                                           2
Res. Buena vista                                         2
Res. Paseo del Prado                                     2
Res. Utila                                               2
Col. Libertad                                            1
Col. Quezaltepec                                         1
Calle Monte Carmelo                                      1
Km 8.5 Troncal del norte                                 1
Residencial Bella Granada                                1
Col. Guadalupe                                           1
Res. Sierra Verde                                        1
Res. Cimas de San Bartolo                                1
Urb. Pinares de Santa Monica                             1
Urb. Cumbres de la Escalon                               1
Zona Alta Col. Escalon                                  

Podemos ver que hay algunos barrios que son iguales pero con nombres diferentes, normalicemos esto.

In [28]:
dataset.loc[dataset['Neighborhood'] == 'Las Camelias en Residencial Las Arboledas ', 'Neighborhood'] = 'Residencial Las Arboledas'
dataset.loc[dataset['Neighborhood'] == 'Metropoli San Gabriel ', 'Neighborhood'] = 'Res. Metropoli San Gabriel '
dataset.loc[dataset['Neighborhood'] == 'Cuidad Marsella', 'Neighborhood'] = 'Res. Cuidad Marsella'
dataset.loc[dataset['Neighborhood'] == 'Col. Miralvalle', 'Neighborhood'] = 'Col. Miralvalle'

In [29]:
dataset['Neighborhood'].value_counts()

Col. Miralvalle                                          3
Res. Villa Constitucion                                  2
Col. Miramonte                                           2
Res. Buena vista                                         2
Res. Paseo del Prado                                     2
Res. Utila                                               2
Col. Libertad                                            1
Col. Quezaltepec                                         1
Calle Monte Carmelo                                      1
Km 8.5 Troncal del norte                                 1
Residencial Bella Granada                                1
Col. Guadalupe                                           1
Res. Sierra Verde                                        1
Res. Cimas de San Bartolo                                1
Urb. Pinares de Santa Monica                             1
Urb. Cumbres de la Escalon                               1
Zona Alta Col. Escalon                                  

In [30]:
#Extraemos los vecindarios con sus coordenadas respectivas en un nuevo set de datos.
neighborhood_data = pd.DataFrame(dataset, columns = ['Neighborhood', 'Latitude', 'Longitude'])
neighborhood_data

,Neighborhood,Latitude,Longitude
0,Col. Libertad,13.7225,-89.2097
1,Calle San Ernesto por Corvisa,13.6814,-89.2011
2,Col. Miramonte,13.7079,-89.2142
3,Res. Buena vista,13.6896,-89.2854
4,Res. Buena vista,13.6896,-89.2854
5,Ciudad Merliot,13.6814,-89.2719
6,Resd. La Gloria por Casa Comunal La Gloria,13.7409,-89.2185
7,Zona del Estadio Cuscatlan,13.6812,-89.2234
9,Col. Miralvalle,13.7213,-89.2227
10,Col. Miralvalle,13.7213,-89.2227


Extraer los lugares cercanos (Supermercados) a cada vecindario y concatenamos estos datos  en un marco de datos.

In [31]:
import requests

url = "https://api.foursquare.com/v3/places/search"

params = {
  	"query": "Supermarket",
  	"ne=13.6893,-89.1871"
  	"sw=13.4883,-89.3222"
  	"open_now": "true",
  	"sort":"DISTANCE"
}

headers = {
    "Accept": "application/json",
    "Authorization": "fsq3PmNUkK/HGoNLTZkiM0BIDkh3pNX86r6AZmdOTyjmG5U="
}

response = requests.request("GET", url, params=params, headers=headers)

print(response.text)

{"results":[{"fsq_id":"4cce034454f0b1f72a0d20ca","categories":[{"id":17069,"name":"Grocery Store / Supermarket","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_","suffix":".png"}}],"chains":[],"distance":492,"geocodes":{"main":{"latitude":13.705074,"longitude":-89.209586}},"link":"/v3/places/4cce034454f0b1f72a0d20ca","location":{"address":"Metrocentro","country":"SV","cross_street":"","formatted_address":"Metrocentro, San Salvador, San Salvador","locality":"San Salvador","region":"San Salvador"},"name":"Super selectos","related_places":{},"timezone":"America/El_Salvador"},{"fsq_id":"4e6f89bc1838a9b627b121df","categories":[{"id":17069,"name":"Grocery Store / Supermarket","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_","suffix":".png"}}],"chains":[],"distance":510,"geocodes":{"main":{"latitude":13.705281,"longitude":-89.209826}},"link":"/v3/places/4e6f89bc1838a9b627b121df","location":{"address":"Blvd. Los Heroes","country":"SV","c

In [32]:
data = {"results":[{"fsq_id":"4cce034454f0b1f72a0d20ca","categories":[{"id":17069,"name":"Grocery Store / Supermarket","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_","suffix":".png"}}],"chains":[],"distance":492,"geocodes":{"main":{"latitude":13.705074,"longitude":-89.209586}},"link":"/v3/places/4cce034454f0b1f72a0d20ca","location":{"address":"Metrocentro","country":"SV","cross_street":"","formatted_address":"Metrocentro, San Salvador, San Salvador","locality":"San Salvador","region":"San Salvador"},"name":"Super selectos","related_places":{},"timezone":"America/El_Salvador"},{"fsq_id":"4e6f89bc1838a9b627b121df","categories":[{"id":17069,"name":"Grocery Store / Supermarket","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_","suffix":".png"}}],"chains":[],"distance":510,"geocodes":{"main":{"latitude":13.705281,"longitude":-89.209826}},"link":"/v3/places/4e6f89bc1838a9b627b121df","location":{"address":"Blvd. Los Heroes","country":"SV","cross_street":"Calle Los Sisimiles","formatted_address":"Blvd. Los Heroes (Calle Los Sisimiles), San Salvador, San Salvador","locality":"San Salvador","region":"San Salvador"},"name":"Super Selectos - Metrocentro 8va Etapa","related_places":{},"timezone":"America/El_Salvador"},{"fsq_id":"4fe62163e4b0bdc4f4739611","categories":[{"id":17069,"name":"Grocery Store / Supermarket","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_","suffix":".png"}}],"chains":[],"distance":724,"geocodes":{"main":{"latitude":13.706652,"longitude":-89.212014}},"link":"/v3/places/4fe62163e4b0bdc4f4739611","location":{"address":"Blvd. Los Heroes","country":"SV","formatted_address":"Blvd. Los Heroes, San Salvador, San Salvador","locality":"San Salvador","region":"San Salvador"},"name":"Super Selectos - Metrocentro 10ma Etapa","related_places":{},"timezone":"America/El_Salvador"},{"fsq_id":"4fc010c7e4b0fddeca884ab6","categories":[{"id":17069,"name":"Grocery Store / Supermarket","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_","suffix":".png"}}],"chains":[],"distance":1010,"geocodes":{"main":{"latitude":13.704269,"longitude":-89.214343}},"link":"/v3/places/4fc010c7e4b0fddeca884ab6","location":{"address":"Blvd. Los Héroes","country":"SV","cross_street":"","formatted_address":"Blvd. Los Héroes, San Salvador, San Salvador","locality":"San Salvador","region":"San Salvador"},"name":"Super Selectos Metrosur","related_places":{},"timezone":"America/El_Salvador"},{"fsq_id":"4d850adb61676dcbf9f480e4","categories":[{"id":17069,"name":"Grocery Store / Supermarket","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_","suffix":".png"}}],"chains":[],"distance":1043,"geocodes":{"main":{"latitude":13.705846,"longitude":-89.214936}},"link":"/v3/places/4d850adb61676dcbf9f480e4","location":{"address":"Boulevar de los Heroes","country":"SV","cross_street":"Frente a Metrosur","formatted_address":"Boulevar de los Heroes (Frente a Metrosur), San Salvador, San Salvador","locality":"San Salvador","region":"San Salvador"},"name":"Despensa de Don Juan","related_places":{"children":[{"fsq_id":"59051e0fdd12f8527b632423","name":"Helados Sarita Miramomtes"}]},"timezone":"America/El_Salvador"},{"fsq_id":"5159c2dee4b077ebbd90b127","categories":[{"id":11000,"name":"Business and Professional Services","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/building/default_","suffix":".png"}}],"chains":[],"distance":1318,"geocodes":{"main":{"latitude":13.707748,"longitude":-89.217452}},"link":"/v3/places/5159c2dee4b077ebbd90b127","location":{"address":"Col Miramonte Avenida Sierra Nevada No 746","country":"SV","cross_street":"","formatted_address":"Col Miramonte Avenida Sierra Nevada No 746, San Salvador, San Salvador","locality":"San Salvador","region":"San Salvador"},"name":"Fu Mau","related_places":{},"timezone":"America/El_Salvador"},{"fsq_id":"4d4de994b1fa236afffa9329","categories":[{"id":17069,"name":"Grocery Store / Supermarket","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_","suffix":".png"}}],"chains":[],"distance":1358,"geocodes":{"main":{"latitude":13.716067,"longitude":-89.21326}},"link":"/v3/places/4d4de994b1fa236afffa9329","location":{"address":"CC San Luis","country":"SV","cross_street":"C. San Antonio Abad","formatted_address":"CC San Luis (C. San Antonio Abad), San Salvador, San Salvador","locality":"San Salvador","region":"San Salvador"},"name":"Super Selectos","related_places":{},"timezone":"America/El_Salvador"},{"fsq_id":"4d55ae3cc6edf04d2773c7a2","categories":[{"id":17069,"name":"Grocery Store / Supermarket","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_","suffix":".png"}}],"chains":[],"distance":1419,"geocodes":{"main":{"latitude":13.699235,"longitude":-89.194558}},"link":"/v3/places/4d55ae3cc6edf04d2773c7a2","location":{"country":"SV","cross_street":"","formatted_address":"San sanvador, San Vicente","locality":"San sanvador","region":"San Vicente"},"name":"Super selectos dario","related_places":{},"timezone":"America/El_Salvador"},{"fsq_id":"5160714ee4b0e63c73daa17a","categories":[{"id":17069,"name":"Grocery Store / Supermarket","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_","suffix":".png"}}],"chains":[],"distance":1425,"geocodes":{"main":{"latitude":13.71844,"longitude":-89.21038}},"link":"/v3/places/5160714ee4b0e63c73daa17a","location":{"country":"SV","cross_street":"","formatted_address":"San Salvador, San Salvador","locality":"San Salvador","region":"San Salvador"},"name":"PriceSmart","related_places":{},"timezone":"America/El_Salvador"},{"fsq_id":"4c6826e77abde21e84e06768","categories":[{"id":17069,"name":"Grocery Store / Supermarket","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_","suffix":".png"}}],"chains":[],"distance":1467,"geocodes":{"main":{"latitude":13.713071,"longitude":-89.193452}},"link":"/v3/places/4c6826e77abde21e84e06768","location":{"address":"27 Calle Poniente","country":"SV","cross_street":"","formatted_address":"27 Calle Poniente, San Salvador, San Salvador","locality":"San Salvador","region":"San Salvador"},"name":"Super Selectos","related_places":{},"timezone":"America/El_Salvador"}],"context":{"geo_bounds":{"circle":{"center":{"latitude":13.7066,"longitude":-89.2053},"radius":22000}}}}
df_dataframe = []
from unicodedata import category
import pandas as pd
result = data['results']
for entry in result:
    name = entry["name"]
    category = entry["categories"][0]['name']
    geocode_dct = entry['geocodes']['main']
    lat = geocode_dct['latitude']
    lon = geocode_dct['longitude']

    df_dataframe.append({
        "name": name,
        "lat": lat,
        "lon": lon,
        "category": category
    })
df = pd.DataFrame(df_dataframe)
print(df)

                                      name        lat        lon  \
0                           Super selectos  13.705074 -89.209586   
1   Super Selectos - Metrocentro 8va Etapa  13.705281 -89.209826   
2  Super Selectos - Metrocentro 10ma Etapa  13.706652 -89.212014   
3                  Super Selectos Metrosur  13.704269 -89.214343   
4                     Despensa de Don Juan  13.705846 -89.214936   
5                                   Fu Mau  13.707748 -89.217452   
6                           Super Selectos  13.716067 -89.213260   
7                     Super selectos dario  13.699235 -89.194558   
8                               PriceSmart  13.718440 -89.210380   
9                           Super Selectos  13.713071 -89.193452   

                             category  
0         Grocery Store / Supermarket  
1         Grocery Store / Supermarket  
2         Grocery Store / Supermarket  
3         Grocery Store / Supermarket  
4         Grocery Store / Supermarket  
5  Business

In [33]:
df.shape

(10, 4)

In [34]:
dataset_venues_market = pd.concat([neighborhood_data, df], axis=1)
dataset_venues_market.head()

,Neighborhood,Latitude,Longitude,name,lat,lon,category
0,Col. Libertad,13.7225,-89.2097,Super selectos,13.705074,-89.209586,Grocery Store / Supermarket
1,Calle San Ernesto por Corvisa,13.6814,-89.2011,Super Selectos - Metrocentro 8va Etapa,13.705281,-89.209826,Grocery Store / Supermarket
2,Col. Miramonte,13.7079,-89.2142,Super Selectos - Metrocentro 10ma Etapa,13.706652,-89.212014,Grocery Store / Supermarket
3,Res. Buena vista,13.6896,-89.2854,Super Selectos Metrosur,13.704269,-89.214343,Grocery Store / Supermarket
4,Res. Buena vista,13.6896,-89.2854,Despensa de Don Juan,13.705846,-89.214936,Grocery Store / Supermarket


In [35]:
dataset_venues_market.shape

(57, 7)

In [36]:
#Corrigiendo error de NaN y rellenando con categoria en la columna de categoria de Supermercados.
dataset_venues_market[dataset_venues_market['category'].str.contains('Supermarket').fillna(False)]

,Neighborhood,Latitude,Longitude,name,lat,lon,category
0,Col. Libertad,13.7225,-89.2097,Super selectos,13.705074,-89.209586,Grocery Store / Supermarket
1,Calle San Ernesto por Corvisa,13.6814,-89.2011,Super Selectos - Metrocentro 8va Etapa,13.705281,-89.209826,Grocery Store / Supermarket
2,Col. Miramonte,13.7079,-89.2142,Super Selectos - Metrocentro 10ma Etapa,13.706652,-89.212014,Grocery Store / Supermarket
3,Res. Buena vista,13.6896,-89.2854,Super Selectos Metrosur,13.704269,-89.214343,Grocery Store / Supermarket
4,Res. Buena vista,13.6896,-89.2854,Despensa de Don Juan,13.705846,-89.214936,Grocery Store / Supermarket
6,Resd. La Gloria por Casa Comunal La Gloria,13.7409,-89.2185,Super Selectos,13.716067,-89.213260,Grocery Store / Supermarket
7,Zona del Estadio Cuscatlan,13.6812,-89.2234,Super selectos dario,13.699235,-89.194558,Grocery Store / Supermarket
9,Col. Miralvalle,13.7213,-89.2227,Super Selectos,13.713071,-89.193452,Grocery Store / Supermarket
8,NaN,NaN,NaN,PriceSmart,13.718440,-89.210380,Grocery Store / Supermarket


Comprobemos cuántas sedes (Supermercados) se han devuelto para cada vecindario.

In [37]:
dataset_venues_market.groupby('Neighborhood').count()

,Latitude,Longitude,name,lat,lon,category
Neighborhood,,,,,,
"700 mts de redondel Ranza, en Res. Paseo del Prado",1,1,0,0,0,0
75 Av. Norte,1,1,0,0,0,0
9a. Calle Pte. Zona Torre Futura,1,1,0,0,0,0
Antiguo Cuscatlan,1,1,0,0,0,0
Av Bernal Calle Castro Moran Villa Victoria 2 Casa #2,1,1,0,0,0,0
Calle Monte Carmelo,1,1,0,0,0,0
Calle San Ernesto por Corvisa,1,1,1,1,1,1
Ciudad Merliot,1,1,1,1,1,1
Col. Escalon,1,1,0,0,0,0


Averigüemos cuántas categorías únicas se pueden crear a partir de todos los lugares devueltos.

In [38]:
print(f'Tenemos {dataset_venues_market["category"].nunique()} categorias unicas.')

Tenemos 2 categorias unicas.


Extraer los lugares cercanos (Supermercados) a cada vecindario y concatenamos estos datos en un marco de datos.

In [39]:
import requests

url = "https://api.foursquare.com/v3/places/search"

params = {
  	"query": "Gym,",
  	"ne=13.6893,-89.1871"
  	"sw=13.4883,-89.3222"
  	"open_now": "true",
  	"sort":"DISTANCE"
}

headers = {
    "Accept": "application/json",
    "Authorization": "fsq3PmNUkK/HGoNLTZkiM0BIDkh3pNX86r6AZmdOTyjmG5U="
}

response = requests.request("GET", url, params=params, headers=headers)

print(response.text)

{"results":[{"fsq_id":"4f92c234e4b0eba9514908d3","categories":[{"id":18021,"name":"Gym and Studio","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/building/gym_","suffix":".png"}}],"chains":[],"distance":672,"geocodes":{"main":{"latitude":13.703405,"longitude":-89.210589}},"link":"/v3/places/4f92c234e4b0eba9514908d3","location":{"country":"SV","cross_street":"","formatted_address":"San Salvador, San Salvador","locality":"San Salvador","region":"San Salvador"},"name":"Perfect Body Gym","related_places":{},"timezone":"America/El_Salvador"},{"fsq_id":"52cf33fb498e8a05da9e97a8","categories":[{"id":18021,"name":"Gym and Studio","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/building/gym_","suffix":".png"}}],"chains":[],"distance":767,"geocodes":{"main":{"latitude":13.712796,"longitude":-89.208447}},"link":"/v3/places/52cf33fb498e8a05da9e97a8","location":{"country":"SV","cross_street":"","formatted_address":""},"name":"Style fitness-gym-studio","related_places":{},"timezon

In [40]:
data = {"results":[{"fsq_id":"62eab5e723020a23e17ee72c","categories":[{"id":18021,"name":"Gym and Studio","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/building/gym_","suffix":".png"}}],"chains":[],"distance":527,"geocodes":{"main":{"latitude":13.705194,"longitude":-89.20997}},"link":"/v3/places/62eab5e723020a23e17ee72c","location":{"address":"Prolongación de Av. Los Andes y Boulevard Tutunichapa Blvd los Héroes, Local258","country":"SV","cross_street":"","formatted_address":"Prolongación de Av. Los Andes y Boulevard Tutunichapa Blvd los Héroes, Local258, San Salvador, San Salvador","locality":"San Salvador","region":"San Salvador"},"name":"Smart Fit","related_places":{"parent":{"fsq_id":"4b648056f964a5209ab82ae3","name":"Centro Comercial Metrocentro"}},"timezone":"America/El_Salvador"},{"fsq_id":"4f92c234e4b0eba9514908d3","categories":[{"id":18021,"name":"Gym and Studio","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/building/gym_","suffix":".png"}}],"chains":[],"distance":672,"geocodes":{"main":{"latitude":13.703405,"longitude":-89.210589}},"link":"/v3/places/4f92c234e4b0eba9514908d3","location":{"country":"SV","cross_street":"","formatted_address":"San Salvador, San Salvador","locality":"San Salvador","region":"San Salvador"},"name":"Perfect Body Gym","related_places":{},"timezone":"America/El_Salvador"},{"fsq_id":"52cf33fb498e8a05da9e97a8","categories":[{"id":18021,"name":"Gym and Studio","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/building/gym_","suffix":".png"}}],"chains":[],"distance":767,"geocodes":{"main":{"latitude":13.712796,"longitude":-89.208447}},"link":"/v3/places/52cf33fb498e8a05da9e97a8","location":{"country":"SV","cross_street":"","formatted_address":""},"name":"Style fitness-gym-studio","related_places":{},"timezone":"America/El_Salvador"},{"fsq_id":"51f7b1be498eb6d6bfa0f080","categories":[{"id":18021,"name":"Gym and Studio","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/building/gym_","suffix":".png"}}],"chains":[],"distance":829,"geocodes":{"main":{"latitude":13.707753,"longitude":-89.212893}},"link":"/v3/places/51f7b1be498eb6d6bfa0f080","location":{"address":"Av. De los Sisimiles","country":"SV","cross_street":"","formatted_address":"Av. De los Sisimiles, San Salvador, San Salvador","locality":"San Salvador","region":"San Salvador"},"name":"Gym @ Real Intercontinental","related_places":{},"timezone":"America/El_Salvador"},{"fsq_id":"4bf2b39582002d7f22d7d0bb","categories":[{"id":19014,"name":"Hotel","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/travel/hotel_","suffix":".png"}}],"chains":[],"distance":842,"geocodes":{"main":{"latitude":13.707502,"longitude":-89.213049}},"link":"/v3/places/4bf2b39582002d7f22d7d0bb","location":{"address":"Boulevard De Los Heroes Y Avenue Sisimiles, Avenue Sisimiles","country":"SV","cross_street":"Avenida Sisimiles","formatted_address":"Boulevard De Los Heroes Y Avenue Sisimiles, Avenue Sisimiles (Avenida Sisimiles), San Salvador, San Salvador","locality":"San Salvador","region":"San Salvador"},"name":"Hotel Real InterContinental San Salvador at Metrocentro Mall","related_places":{"children":[{"fsq_id":"5565d31a498eb08d52839c3e","name":"The Coffee Corner"},{"fsq_id":"4d8245517465cbff7c81cbed","name":"Faisca do Brasil Rodizio & Grill"},{"fsq_id":"5365be08498ea0b7ddfd2f6b","name":"Nau Lounge"},{"fsq_id":"52725c8511d213753a19d19f","name":"Ambassador Floor, InterContinental"},{"fsq_id":"588d1be0ac136914a69a22a8","name":"Salon VIP Hotel Real InterContinental"},{"fsq_id":"531a7bca498e88c061dd2d7f","name":"Picasso"}]},"timezone":"America/El_Salvador"},{"fsq_id":"51b76d89498e4d483476ee1f","categories":[{"id":12027,"name":"College Gym","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/building/gym_","suffix":".png"}}],"chains":[],"distance":862,"geocodes":{"main":{"latitude":13.699689,"longitude":-89.201673}},"link":"/v3/places/51b76d89498e4d483476ee1f","location":{"country":"SV","cross_street":"","formatted_address":"San Salvador, San Salvador","locality":"San Salvador","region":"San Salvador"},"name":"polideportivo Utec","related_places":{},"timezone":"America/El_Salvador"},{"fsq_id":"528037ea11d2c5390e8f2290","categories":[{"id":12027,"name":"College Gym","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/building/gym_","suffix":".png"}}],"chains":[],"distance":908,"geocodes":{"main":{"latitude":13.700404,"longitude":-89.199811}},"link":"/v3/places/528037ea11d2c5390e8f2290","location":{"address":"17 Ave. Nte.","country":"SV","cross_street":"Calle Arce","formatted_address":"17 Ave. Nte. (Calle Arce), 00000-San Salvador, San Salvador","locality":"San Salvador","postcode":"00000","region":"San Salvador"},"name":"UTEC Fitness Gym","related_places":{},"timezone":"America/El_Salvador"},{"fsq_id":"53b1fb23498e1978c88c5ee5","categories":[{"id":18030,"name":"Gymnastics Center","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/building/gym_","suffix":".png"}}],"chains":[],"distance":939,"geocodes":{"main":{"latitude":13.704639,"longitude":-89.213763}},"link":"/v3/places/53b1fb23498e1978c88c5ee5","location":{"address":"Centro Comercial Metrocentro","country":"SV","cross_street":"Boulevard Los Héroes","formatted_address":"Centro Comercial Metrocentro (Boulevard Los Héroes), San Salvador, San Salvador","locality":"San Salvador","region":"San Salvador"},"name":"Be Fit","related_places":{},"timezone":"America/El_Salvador"},{"fsq_id":"50c27157e4b00ce4a3b66ce6","categories":[{"id":18000,"name":"Sports and Recreation","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/shops/sports_outdoors_","suffix":".png"}}],"chains":[],"distance":1073,"geocodes":{"main":{"latitude":13.697986,"longitude":-89.2098}},"link":"/v3/places/50c27157e4b00ce4a3b66ce6","location":{"address":"6a 10a Calle Poniente. costado sur gimnasio nacional","country":"SV","cross_street":"Costado Sur Gimnasio Nacional","formatted_address":"6a 10a Calle Poniente. costado sur gimnasio nacional (Costado Sur Gimnasio Nacional), San Salvador, San Salvadot","locality":"San Salvador","region":"San Salvadot"},"name":"Federacion Salvadorena de Montanismo y Escalada (FSME)","related_places":{},"timezone":"America/El_Salvador"},{"fsq_id":"4f92c325e4b00fe65e93f48c","categories":[{"id":18021,"name":"Gym and Studio","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/building/gym_","suffix":".png"}}],"chains":[],"distance":1155,"geocodes":{"main":{"latitude":13.715815,"longitude":-89.210261}},"link":"/v3/places/4f92c325e4b00fe65e93f48c","location":{"country":"SV","cross_street":"","formatted_address":"San Salvador, San Salvador","locality":"San Salvador","region":"San Salvador"},"name":"Perfect Body Gym","related_places":{},"timezone":"America/El_Salvador"}],"context":{"geo_bounds":{"circle":{"center":{"latitude":13.7066,"longitude":-89.2053},"radius":22000}}}}
df_dataframe1 = []
from unicodedata import category
import pandas as pd
result = data['results']
for entry in result:
    name = entry["name"]
    category = entry["categories"][0]['name']
    geocode_dct = entry['geocodes']['main']
    lat = geocode_dct['latitude']
    lon = geocode_dct['longitude']

    df_dataframe1.append({
        "name": name,
        "lat": lat,
        "lon": lon,
        "category": category
    })
df1 = pd.DataFrame(df_dataframe1)
print(df1)

                                                name        lat        lon  \
0                                          Smart Fit  13.705194 -89.209970   
1                                   Perfect Body Gym  13.703405 -89.210589   
2                           Style fitness-gym-studio  13.712796 -89.208447   
3                        Gym @ Real Intercontinental  13.707753 -89.212893   
4  Hotel Real InterContinental San Salvador at Me...  13.707502 -89.213049   
5                                 polideportivo Utec  13.699689 -89.201673   
6                                   UTEC Fitness Gym  13.700404 -89.199811   
7                                             Be Fit  13.704639 -89.213763   
8  Federacion Salvadorena de Montanismo y Escalad...  13.697986 -89.209800   
9                                   Perfect Body Gym  13.715815 -89.210261   

                category  
0         Gym and Studio  
1         Gym and Studio  
2         Gym and Studio  
3         Gym and Studio  
4     

In [41]:
df1.shape

(10, 4)

In [42]:
#Creando un nuevo dataset para validar los locales cercanos Gimnasios.
dataset_venues_gym = pd.concat([neighborhood_data,df1], axis=1)
dataset_venues_gym.head()

,Neighborhood,Latitude,Longitude,name,lat,lon,category
0,Col. Libertad,13.7225,-89.2097,Smart Fit,13.705194,-89.209970,Gym and Studio
1,Calle San Ernesto por Corvisa,13.6814,-89.2011,Perfect Body Gym,13.703405,-89.210589,Gym and Studio
2,Col. Miramonte,13.7079,-89.2142,Style fitness-gym-studio,13.712796,-89.208447,Gym and Studio
3,Res. Buena vista,13.6896,-89.2854,Gym @ Real Intercontinental,13.707753,-89.212893,Gym and Studio
4,Res. Buena vista,13.6896,-89.2854,Hotel Real InterContinental San Salvador at Me...,13.707502,-89.213049,Hotel


In [43]:
dataset_venues_gym.shape

(57, 7)

In [110]:
#Corrigiendo error de NaN y rellenando con categoria en la columna de categoria de Gimnasios.
dataset_venues_gym[dataset_venues_gym['category'].str.contains('Gym and Studio').fillna(False)]

,Neighborhood,Latitude,Longitude,name,lat,lon,category
0,Col. Libertad,13.7225,-89.2097,Smart Fit,13.705194,-89.209970,Gym and Studio
1,Calle San Ernesto por Corvisa,13.6814,-89.2011,Perfect Body Gym,13.703405,-89.210589,Gym and Studio
2,Col. Miramonte,13.7079,-89.2142,Style fitness-gym-studio,13.712796,-89.208447,Gym and Studio
3,Res. Buena vista,13.6896,-89.2854,Gym @ Real Intercontinental,13.707753,-89.212893,Gym and Studio
9,Col. Miralvalle,13.7213,-89.2227,Perfect Body Gym,13.715815,-89.210261,Gym and Studio


Comprobemos cuántas sedes (Gimnasio) se han devuelto para cada vecindario.

In [44]:
dataset_venues_gym.groupby('Neighborhood').count()

,Latitude,Longitude,name,lat,lon,category
Neighborhood,,,,,,
"700 mts de redondel Ranza, en Res. Paseo del Prado",1,1,0,0,0,0
75 Av. Norte,1,1,0,0,0,0
9a. Calle Pte. Zona Torre Futura,1,1,0,0,0,0
Antiguo Cuscatlan,1,1,0,0,0,0
Av Bernal Calle Castro Moran Villa Victoria 2 Casa #2,1,1,0,0,0,0
Calle Monte Carmelo,1,1,0,0,0,0
Calle San Ernesto por Corvisa,1,1,1,1,1,1
Ciudad Merliot,1,1,1,1,1,1
Col. Escalon,1,1,0,0,0,0


Averigüemos cuántas categorías únicas se pueden crear a partir de todos los lugares devueltos.

In [45]:
print(f'Tenemos {dataset_venues_gym["category"].nunique()} categorias unicas.')

Tenemos 5 categorias unicas.


Ahora que ya tenemos nuestras sedes, vamos a ver las que nos interesan: gimnasios, supermercados.

Primero vamos a ver nuestras sedes de Gimnasios

In [46]:
dataset_gym = pd.read_csv('gym.csv')
print('Set de datos cargado con exito!')

Set de datos cargado con exito!


In [47]:
dataset_gym.head()

,Codigo,Neighborhood,Latitude,Longitude,name,lat,lon,category
0,0,Col. Libertad,13.7225,-89.2097,Smart Fit,13.705194,-89.209970,Gym and Studio
1,1,Calle San Ernesto por Corvisa,13.6814,-89.2011,Perfect Body Gym,13.703405,-89.210589,Gym and Studio
2,2,Col. Miramonte,13.7079,-89.2142,Style fitness-gym-studio,13.712796,-89.208447,Gym and Studio
3,3,Res. Buena vista,13.6896,-89.2854,Gym @ Real Intercontinental,13.707753,-89.212893,Gym and Studio
4,4,Res. Buena vista,13.6896,-89.2854,Hotel Real InterContinental San Salvador at Me...,13.707502,-89.213049,Hotel


In [48]:
gym_venues = dataset_gym.loc[dataset_gym['category'].str.contains('Gym')]
print(f'Hay en total {gym_venues.shape[0]} Gimnasios')

Hay en total 22 Gimnasios


Segundo vamos a ver nuestras sedes de supermercados.

In [49]:
dataset_market = pd.read_csv('market.csv')
print('Set de datos cargado con exito!')

Set de datos cargado con exito!


In [50]:
dataset_market.head()

,codigo,Neighborhood,Latitude,Longitude,name,lat,lon,category
0,0,Col. Libertad,13.7225,-89.2097,Super selectos,13.705074,-89.209586,Grocery Store / Supermarket
1,1,Calle San Ernesto por Corvisa,13.6814,-89.2011,Super Selectos - Metrocentro 8va Etapa,13.705281,-89.209826,Grocery Store / Supermarket
2,2,Col. Miramonte,13.7079,-89.2142,Super Selectos - Metrocentro 10ma Etapa,13.706652,-89.212014,Grocery Store / Supermarket
3,3,Res. Buena vista,13.6896,-89.2854,Super Selectos Metrosur,13.704269,-89.214343,Grocery Store / Supermarket
4,4,Res. Buena vista,13.6896,-89.2854,Despensa de Don Juan,13.705846,-89.214936,Grocery Store / Supermarket


In [51]:
market_venues = dataset_market.loc[dataset_market['category'].str.contains('Supermarket')]
print(f'Hay en total {market_venues.shape[0]} Supermercados')

Hay en total 40 Supermercados


Para finalizar la sección de datos, añadamos a nuestro mapa anterior todos los locales de las categorías anteriores.

In [72]:
# Agregando los Supermercados a nuestro mapa.
for lat, lng, label in zip(market_venues['lat'], market_venues['lon'], market_venues['name']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=f'{label}',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
).add_to(map_city)
    
# Agregando los Gimnasios a nuestro mapa.
for lat, lng, label in zip(gym_venues['Latitude'], gym_venues['Longitude'], gym_venues['name']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=f'{label}',
        fill=True,
        color='green',
        fill_color='green',
        fill_opacity=0.6
).add_to(map_city)

map_city

<h2>Metodologia.</h2> <a name="metodologia"></a>

En este proyecto tenemos que encontrar una casa, una habitacion o un apartamento que se ajuste a nuestras necesidades.

En el primer paso hemos recopilado los datos necesarios: datos de propiedades de alquiler, tipo (categoría) de cada local en un radio de 1 km de cada vecindario donde se conoce de alquileres. También hemos filtrado todos los alquileres de (Casa, Habitacion y Apartamento) en un radio de 15 km desde el lugar de trabajo.

En el segundo paso realizaremos un análisis exploratorio de los datos para extraer información sobre los datos económicos y sociales de cada vecindario y realizaremos un k-means para agrupar los vecindarios.

Calcularemos la distancia entre cada local seleccionado (gimnasios y mercados) y cada apartamento, para encontrar el local de cada categoría más cercano a cada casa , habitacion y apartamento.

Por último, filtraremos los datos para adaptarlos a nuestras necesidades básicas, como el valor del alquiler, la distancia a los locales y el número de habitaciones.

<h2>Analisis.</h2> <a name="analisis"></a>

Analicemos primero cada vecindario y averigüemos más sobre cada uno de ellos.

In [142]:
#Crearemos un dataset nuevo donde integraremos categorias.
dataset_venues_concat = pd.concat([dataset_venues_gym,dataset_venues_market], axis=1)
dataset_venues_concat.head()

,Neighborhood,Latitude,Longitude,name,lat,lon,category,Neighborhood,Latitude,Longitude,name,lat,lon,category
0,Col. Libertad,13.7225,-89.2097,Smart Fit,13.705194,-89.209970,Gym and Studio,Col. Libertad,13.7225,-89.2097,Super selectos,13.705074,-89.209586,Grocery Store / Supermarket
1,Calle San Ernesto por Corvisa,13.6814,-89.2011,Perfect Body Gym,13.703405,-89.210589,Gym and Studio,Calle San Ernesto por Corvisa,13.6814,-89.2011,Super Selectos - Metrocentro 8va Etapa,13.705281,-89.209826,Grocery Store / Supermarket
2,Col. Miramonte,13.7079,-89.2142,Style fitness-gym-studio,13.712796,-89.208447,Gym and Studio,Col. Miramonte,13.7079,-89.2142,Super Selectos - Metrocentro 10ma Etapa,13.706652,-89.212014,Grocery Store / Supermarket
3,Res. Buena vista,13.6896,-89.2854,Gym @ Real Intercontinental,13.707753,-89.212893,Gym and Studio,Res. Buena vista,13.6896,-89.2854,Super Selectos Metrosur,13.704269,-89.214343,Grocery Store / Supermarket
4,Res. Buena vista,13.6896,-89.2854,Hotel Real InterContinental San Salvador at Me...,13.707502,-89.213049,Hotel,Res. Buena vista,13.6896,-89.2854,Despensa de Don Juan,13.705846,-89.214936,Grocery Store / Supermarket


In [156]:
# one hot encoding
neighborhood_onehot = pd.get_dummies(dataset_venues_concat[['category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
#neighborhood_onehot['Neighborhood'] = dataset_venues_concat['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neighborhood_onehot.columns[-1]] + list(neighborhood_onehot.columns[:-1])
neighborhood_onehot = neighborhood_onehot[fixed_columns]

neighborhood_onehot.head()

,Grocery Store / Supermarket,College Gym,Gym and Studio,Gymnastics Center,Hotel,Sports and Recreation,Business and Professional Services
0,1,0,1,0,0,0,0
1,1,0,1,0,0,0,0
2,1,0,1,0,0,0,0
3,1,0,1,0,0,0,0
4,1,0,0,0,1,0,0


A continuación, agrupemos las filas por vecindarios y tomando la media de la frecuencia de aparición de cada categoría

In [153]:
print(f'Tenemos {neighborhood_onehot.shape[0]} vecindarios y {neighborhood_onehot.shape[1]} sedes por categorias.')

Tenemos 57 vecindarios y 7 sedes por categorias.


Ahora vamos a crear un nuevo marco de datos y mostrar los locales principales de cada vecindario para conocer mejor las características sociales.

En primer lugar, vamos a escribir una función para ordenar las sedes en orden descendente.

In [114]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]
print('Funcion ejecutada!')

Funcion ejecutada!


Y ahora, crearemos el marco de datos.

Ahora vamos a agrupar los datos económicos de cada vecindario. Primero vamos a obtener el tamaño de cada vecindario para saber si tiene suficientes registros para ser tenido en cuenta.

In [171]:
#Vamos a cargar un dataframe.
dataset = pd.read_csv('economic_real_state.csv')
print('Set de datos cargado con exito!')

Set de datos cargado con exito!


In [172]:
dataset.head()

,tipo_alquiler,habitaciones,precio_alquiler,neighborhood,Municipio,Departamento,Latitud,Longitud
0,Apartamento,1,450,Col. Libertad,San Salvador,San Salvador,13.7225,-89.2097
1,Habitacion,1,90,Calle San Ernesto por Corvisa,San Salvador,San Salvador,13.6814,-89.2011
2,Habitacion,1,190,Col. Miramonte,San Salvador,San Salvador,13.7079,-89.2142
3,Habitacion,1,200,Res. Buena vista,Santa Tecla,La Libertad,13.6896,-89.2854
4,Apartamento,1,400,Res. Buena vista,Santa Tecla,La Libertad,13.6896,-89.2854


In [139]:
dataset.shape

(102, 8)

In [173]:
#Crearemos un dataset nuevo donde integraremos vecindarios y categorias para ver el nivel economico de los vecindarios.
dataset_venues_concat = pd.concat([dataset, neighborhood_onehot], axis=1)
dataset_venues_concat.head()

,tipo_alquiler,habitaciones,precio_alquiler,neighborhood,Municipio,Departamento,Latitud,Longitud,Grocery Store / Supermarket,College Gym,Gym and Studio,Gymnastics Center,Hotel,Sports and Recreation,Business and Professional Services
0,Apartamento,1,450,Col. Libertad,San Salvador,San Salvador,13.7225,-89.2097,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Habitacion,1,90,Calle San Ernesto por Corvisa,San Salvador,San Salvador,13.6814,-89.2011,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Habitacion,1,190,Col. Miramonte,San Salvador,San Salvador,13.7079,-89.2142,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,Habitacion,1,200,Res. Buena vista,Santa Tecla,La Libertad,13.6896,-89.2854,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,Apartamento,1,400,Res. Buena vista,Santa Tecla,La Libertad,13.6896,-89.2854,1.0,0.0,0.0,0.0,1.0,0.0,0.0


Ahora vamos a agrupar los datos económicos de cada vecindario. Primero vamos a obtener el tamaño de cada vecindario para saber si tiene suficientes registros para ser tenido en cuenta.

In [174]:
neighborhood_economic = dataset.groupby('neighborhood').mean()[['precio_alquiler', 'habitaciones']].reset_index()
neighborhood_economic['tamaño'] = dataset.groupby('neighborhood').size().values

C:\Users\cristian_parada\AppData\Local\Temp\ipykernel_19852\334354024.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  neighborhood_economic = dataset.groupby('neighborhood').mean()[['precio_alquiler', 'habitaciones']].reset_index()


In [175]:
neighborhood_economic.head()

,neighborhood,precio_alquiler,habitaciones,tamaño
0,"700 mts de redondel Ranza, en Res. Paseo del P...",750.0,3.0,1
1,75 Av. Norte,650.0,3.0,1
2,9a. Calle Pte. Zona Torre Futura,850.0,2.0,1
3,Antiguo Cuscatlan,300.0,1.0,1
4,Av Bernal Calle Castro Moran Villa Victoria 2 ...,600.0,3.0,1


In [176]:
#Crearemos un dataset nuevo donde integraremos vecindarios y categorias para ver el nivel economico de los vecindarios.
neighborhood_social_economic = pd.concat([neighborhood_economic, neighborhood_onehot], axis=1)

In [177]:
neighborhood_social_economic.head()

,neighborhood,precio_alquiler,habitaciones,tamaño,Grocery Store / Supermarket,College Gym,Gym and Studio,Gymnastics Center,Hotel,Sports and Recreation,Business and Professional Services
0,"700 mts de redondel Ranza, en Res. Paseo del P...",750.0,3.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,75 Av. Norte,650.0,3.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,9a. Calle Pte. Zona Torre Futura,850.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,Antiguo Cuscatlan,300.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,Av Bernal Calle Castro Moran Villa Victoria 2 ...,600.0,3.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [185]:
#Quitar los valores NaN de nuestro data set previamente a crear los cluster.
neighborhood_social_economic.fillna(0, inplace=True)

Ahora, vamos a ejecutar un k-means para agrupar nuestros vecindarios.

In [186]:
# set number of clusters
kclusters = 5

neighborhood_grouped_clustering = neighborhood_social_economic.drop('neighborhood', 1)

# run k-means clusteringneighborhood_social_economic
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhood_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

C:\Users\cristian_parada\AppData\Local\Temp\ipykernel_19852\1361847076.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  neighborhood_grouped_clustering = neighborhood_social_economic.drop('neighborhood', 1)
C:\Users\cristian_parada\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([4, 0, 4, 3, 0, 3, 3, 2, 1, 3])

Creemos un nuevo marco de datos que incluya la agrupación y los  principales locales de cada vecindario.

In [187]:


# add clustering labels
neighborhood_social_economic.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhood_merged = neighborhood_data

# merge neighborhood_data with neighborhood_social_economic to add latitude/longitude for each neighborhood
neighborhood_merged = neighborhood_merged.merge(neighborhood_social_economic, left_on='Neighborhood', right_on='neighborhood').drop(columns=['neighborhood'])

neighborhood_merged.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,precio_alquiler,habitaciones,tamaño,Grocery Store / Supermarket,College Gym,Gym and Studio,Gymnastics Center,Hotel,Sports and Recreation,Business and Professional Services
0,Col. Libertad,13.7225,-89.2097,3,450.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Calle San Ernesto por Corvisa,13.6814,-89.2011,2,90.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,Col. Miramonte,13.7079,-89.2142,0,545.0,2.5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Col. Miramonte,13.7077,-89.2138,0,545.0,2.5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Res. Buena vista,13.6896,-89.2854,3,300.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Por último, visualicemos los clusters resultantes

In [188]:
# create map
map_clusters = folium.Map(location=[work_location[0], work_location[1]], zoom_start=14)

colors=['red', 'yellow', 'orange', 'green', 'blue']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhood_merged['Latitude'], neighborhood_merged['Longitude'], neighborhood_merged['Neighborhood'], neighborhood_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors[cluster-1],
        fill=True,
        fill_color=colors[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Veamos cuántos registros hay en cada clúster.

In [189]:
for cluster in np.sort(neighborhood_merged['Cluster Labels'].unique()):
    print(f'There are {neighborhood_merged.loc[neighborhood_merged["Cluster Labels"] == cluster].shape[0]} registers on Cluster {cluster}')

There are 20 registers on Cluster 0
There are 10 registers on Cluster 1
There are 2 registers on Cluster 2
There are 11 registers on Cluster 3
There are 13 registers on Cluster 4


<h4>CLUSTER # 1</h4>

In [190]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 0, neighborhood_merged.columns[[0] + list(range(4, neighborhood_merged.shape[1]))]]

,Neighborhood,precio_alquiler,habitaciones,tamaño,Grocery Store / Supermarket,College Gym,Gym and Studio,Gymnastics Center,Hotel,Sports and Recreation,Business and Professional Services
2,Col. Miramonte,545.0,2.500000,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Col. Miramonte,545.0,2.500000,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Col. Miralvalle,525.0,2.333333,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,Col. Miralvalle,525.0,2.333333,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,Col. Miralvalle,525.0,2.333333,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,"Colonia San Francisco,",550.0,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,Colonia La Rabida,600.0,4.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,75 Av. Norte,650.0,3.000000,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
21,Las Camelias en Residencial Las Arboledas,580.0,3.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22,Res. Alpes Suizos,500.0,3.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h4>CLUSTER # 2</h4>

In [191]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 1, neighborhood_merged.columns[[0] + list(range(4, neighborhood_merged.shape[1]))]]

,Neighborhood,precio_alquiler,habitaciones,tamaño,Grocery Store / Supermarket,College Gym,Gym and Studio,Gymnastics Center,Hotel,Sports and Recreation,Business and Professional Services
7,Resd. La Gloria por Casa Comunal La Gloria,140.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Zona del Estadio Cuscatlan,150.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,Col. San Francisco por el Estadio Cuscatlan,200.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,Col. Escalon,250.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,Pasaje en Calle La Cañada Ciudad Merliot,225.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,Condominio Villas de San Francisco,250.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,Col. Guadalupe,185.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41,Res. Villa Constitucion,287.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42,Res. Villa Constitucion,287.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49,Urb. Santisima Trinidad 2a. Etapa,225.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h4>CLUSTER # 3</h4>

In [192]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 2, neighborhood_merged.columns[[0] + list(range(4, neighborhood_merged.shape[1]))]]

,Neighborhood,precio_alquiler,habitaciones,tamaño,Grocery Store / Supermarket,College Gym,Gym and Studio,Gymnastics Center,Hotel,Sports and Recreation,Business and Professional Services
1,Calle San Ernesto por Corvisa,90.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
18,km 11 carretera Troncal del norte Col San Anto...,100.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h4>CLUSTER # 4</h4>

In [194]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 3, neighborhood_merged.columns[[0] + list(range(4, neighborhood_merged.shape[1]))]]

,Neighborhood,precio_alquiler,habitaciones,tamaño,Grocery Store / Supermarket,College Gym,Gym and Studio,Gymnastics Center,Hotel,Sports and Recreation,Business and Professional Services
0,Col. Libertad,450.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Res. Buena vista,300.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Res. Buena vista,300.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Ciudad Merliot,300.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,Antiguo Cuscatlan,300.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
29,Res. Cimas de San Bartolo III,350.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32,Resd. La Cima 4,375.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,Calle Monte Carmelo,350.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
35,Km 8.5 Troncal del norte,375.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36,Residencial Bella Granada,380.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h4>CLUSTER # 5</h4>

In [195]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 4, neighborhood_merged.columns[[0] + list(range(4, neighborhood_merged.shape[1]))]]

,Neighborhood,precio_alquiler,habitaciones,tamaño,Grocery Store / Supermarket,College Gym,Gym and Studio,Gymnastics Center,Hotel,Sports and Recreation,Business and Professional Services
24,"700 mts de redondel Ranza, en Res. Paseo del P...",750.0,3.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
38,Res. Sierra Verde,710.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,Urb. Pinares de Santa Monica,750.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43,Zona Alta Col. Escalon,700.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,Res. Utila,850.0,2.5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45,Res. Utila,850.0,2.5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46,Urb. Cumbres de la Escalon,850.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47,Col. San Benito,850.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,Col. Escalon Parte Baja,725.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50,Res. Fincas de Asturias,850.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Ahora que sabemos más sobre nuestros vecindarios, vamos a calcular el lugar específico más cercano (gimnasios y supermercados) a cada lugar de alquiler de nuestro conjunto de datos.

Primero vamos a fusionar la latitud y la longitud en una sola fila para utilizar la biblioteca haversine.

In [197]:
gym_venues = get_location(gym_venues, name_latitude='Latitude', name_longitude='Longitude')
market_venues = get_location(market_venues, name_latitude='Latitude', name_longitude='Longitude')

gym_venues.reset_index(inplace=True)
market_venues.reset_index(inplace=True)

In [198]:
gym_venues.head()

,index,Codigo,Neighborhood,Latitude,Longitude,name,lat,lon,category,location
0,0,0,Col. Libertad,13.7225,-89.2097,Smart Fit,13.705194,-89.209970,Gym and Studio,"[13.7225, -89.2097]"
1,1,1,Calle San Ernesto por Corvisa,13.6814,-89.2011,Perfect Body Gym,13.703405,-89.210589,Gym and Studio,"[13.6814, -89.2011]"
2,2,2,Col. Miramonte,13.7079,-89.2142,Style fitness-gym-studio,13.712796,-89.208447,Gym and Studio,"[13.7079, -89.2142]"
3,3,3,Res. Buena vista,13.6896,-89.2854,Gym @ Real Intercontinental,13.707753,-89.212893,Gym and Studio,"[13.6896, -89.2854]"
4,5,5,Ciudad Merliot,13.6814,-89.2719,polideportivo Utec,13.699689,-89.201673,College Gym,"[13.6814, -89.2719]"


In [199]:
market_venues.head()

,index,codigo,Neighborhood,Latitude,Longitude,name,lat,lon,category,location
0,0,0,Col. Libertad,13.7225,-89.2097,Super selectos,13.705074,-89.209586,Grocery Store / Supermarket,"[13.7225, -89.2097]"
1,1,1,Calle San Ernesto por Corvisa,13.6814,-89.2011,Super Selectos - Metrocentro 8va Etapa,13.705281,-89.209826,Grocery Store / Supermarket,"[13.6814, -89.2011]"
2,2,2,Col. Miramonte,13.7079,-89.2142,Super Selectos - Metrocentro 10ma Etapa,13.706652,-89.212014,Grocery Store / Supermarket,"[13.7079, -89.2142]"
3,3,3,Res. Buena vista,13.6896,-89.2854,Super Selectos Metrosur,13.704269,-89.214343,Grocery Store / Supermarket,"[13.6896, -89.2854]"
4,4,4,Res. Buena vista,13.6896,-89.2854,Despensa de Don Juan,13.705846,-89.214936,Grocery Store / Supermarket,"[13.6896, -89.2854]"


<h2>Resultado y Debate.</h2> <a name="resultados"></a>

Nuestro análisis muestra que, aunque hay un gran número de apartamentos menores de (500 USD) en nuestro conjunto de datos original, sólo unos pocos (17) se ajustan a nuestro perfil. Nuestro primer filtro con el objetivo de obtener sólo alquileres en un radio de 15 km para nuestro lugar de trabajo no filtró muchas propiedades (36) y eso se debe a que ya habíamos filtrado los vecindarios para validar los datos.

El mapa de calor muestra que hay una clara concentración de los alquileres en los vecindarios que filtramos antes, lo que resulta en un pequeño sesgo en nuestros resultados. Para los próximos proyectos podríamos considerar obtener todos los vecindarios en un radio de 5 km desde nuestro punto de partida pero con la condicion de aumentar el valor de alquiler puesto que en un radio mas cercano hay pero a un costo mucho mayor al que se tiene para el ejercicio.

Después de agrupar los datos económicos y sociales de los vecindarios, los agrupamos en 5 grupos. se puede ver que, los vecindarios no son muy similares entre sí, lo que resulta en una alta concentración en uno de los grupos (~34% de los vecindarios fuera del radio de los 15 kms).

Para encontrar nuestros alquileres potenciales, aplicamos los filtros finales considerados como número de dormitorios, distancia mínima a lugares específicos y valor máximo de alquiler mensual. Los filtros fueron muy eficaces, filtrando aproximadamente el 99,4% de los resultados.

Para generar nuestro conjunto de datos final, agrupamos los datos de cada alquiler con la información de los locales de interés más cercanos y los datos económicos y sociales del vecindario. Esto nos proporcionó una visión amplia de las características de cada apartamento y su entorno, cabe destacar que como tal el pais no esta soportado por Foursquare y el dato de barrios o vecindarios es escaso pues en en Salvador se toman como municipios o lugares de alta concentracion poblacional no delimitados solo conocidos popularmente.

<h2>Conclusion.</h2> <a name="conclusion"></a>

El objetivo de este proyecto era encontrar el mejor alquiler de casa, habitacion o apartamento  en alquiler en San Salvador y sus alrededores (Gran San Salvador) que se ajustara a nuestro perfil. Reuniendo datos de casas, habitaciones y apartamentos en vecindarios populosos cercanos y extrayendo los datos de estos vecindarios, hemos generado un conjunto de datos concentrado con los alquileres más prometedores para vivir.

La decisión final debe tomarse siempre con cautela, después de examinar cada alquiler individualmente. El proyecto sirvió para filtrar el gran número de alquileres y agrupar los datos de cada alquiler potencial y los datos sociales y económicos de su respectivo vecindario. Los datos economicos, sociales,  calidad de vida,  delincuencia, movilidad urbana, etc. no se tuvieron en cuenta y podrían ser indicadores interesantes a tener en cuenta en próximos proyectos para mejorar el análisis, ademas de que se debe de tener o construir un set de datos robusto pues actualmente no existe como tal de ninguna institucion publica o privada del pais sobre este tema.
